In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
from wrf import getvar
import celer
from scipy.ndimage import gaussian_filter
import gc
%matplotlib inline

sys.path.insert(1, '/work/08350/tg876493/stampede2/python_codes/2020_TC_CRF/dev/freddy0218/tools/')
import derive_var
import myeof 
import read_and_proc
from derive_var import forward_diff

plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

#### PCA members needed (Urad:14,Vtan:16,w:40)

In [2]:
############################################################################################################################################
# PCA_dict: Processed EOF objects; A_dict: Variable anomaly matrices; Af_dict: Flattened variable anomaly matrices
# *note that following EOF conventions, we derived EOFs with anomalies.
############################################################################################################################################
import pickle
from tqdm import tqdm
path = '/scratch/08350/tg876493/ruppert2020/output/'
pca_dict,flatvar_dict = {},{}
varname = ['U','V','W','QV','dthdt']
for inx,obj in tqdm(enumerate(varname)):
    pca_dict[varname[inx]] = read_and_proc.depickle(path+'pca/'+str(obj)+'_pca')
    flatvar_dict[varname[inx]] = read_and_proc.depickle(path+'flatvar/'+str(obj)+'_flatvar')
#dudvdw_dict = {}
#varname = ['dw']#,'dv','dw']
#for inx,obj in tqdm(enumerate(varname)):
 #   dudvdw_dict[varname[inx]] = read_and_proc.depickle(path+'dudvdw/'+str(obj)+'_flatvar')

5it [01:49, 22.00s/it]


In [8]:
print("The number of required components to explain 90% variance of each variables: ")
print("dthetadt:"+str(np.abs(pca_dict['dthdt'].explained_variance_ratio_.cumsum()-0.8).argmin())+" "\
      "urad:"+str(np.abs(pca_dict['U'].explained_variance_ratio_.cumsum()-0.8).argmin())+" "\
      "vtan:"+str(np.abs(pca_dict['V'].explained_variance_ratio_.cumsum()-0.8).argmin())+" "\
      "w:"+str(np.abs(pca_dict['W'].explained_variance_ratio_.cumsum()-0.8).argmin())+" "\
     "qv:"+str(np.abs(pca_dict['QV'].explained_variance_ratio_.cumsum()-0.8).argmin()))

The number of required components to explain 90% variance of each variables: 
dthetadt:11 urad:3 vtan:1 w:20 qv:3


##### Results

In [7]:
read_and_proc.depickle('./Vtan_var_new')

[0.0,
 0.2349252,
 0.31573617,
 0.4522541,
 0.5136231,
 0.5648523,
 0.6347567,
 0.6814561,
 0.70532167,
 0.73693025,
 0.7583128,
 0.7770223,
 0.7949878,
 0.8103021]

In [7]:
for i in np.linspace(0,26,14):
    print(i)

0.0
2.0
4.0
6.0
8.0
10.0
12.0
14.0
16.0
18.0
20.0
22.0
24.0
26.0


In [3]:
def output_varexplained(varname='U',dudvdw_name='du',pca_dict=None,flatvar_dict=None,dudvdw_dict=None,member=np.linspace(0,25,26)):
    output = []
    for i in (member):
        temp1 = np.dot(forward_diff(pca_dict[varname].transform(flatvar_dict[varname])[0:108,0:int(i)],60*60,0,1),(pca_dict[varname].components_[0:int(i)]))
        temp2 = np.dot(forward_diff(pca_dict[varname].transform(flatvar_dict[varname])[108:108+96,0:int(i)],60*60,0,1),(pca_dict[varname].components_[0:int(i)]))
        temp3 = np.dot(forward_diff(pca_dict[varname].transform(flatvar_dict[varname])[108+96:108+96+72,0:int(i)],60*60,0,1),(pca_dict[varname].components_[0:int(i)]))
        temp4 = np.dot(forward_diff(pca_dict[varname].transform(flatvar_dict[varname])[108+96+72:108+96+72+96,0:int(i)],60*60,0,1),(pca_dict[varname].components_[0:int(i)]))
        temp = np.concatenate((temp1,temp2,temp3,temp4),axis=0)
        del temp1,temp2,temp3,temp4
        gc.collect()
        
        toout = np.var(temp)/np.var(dudvdw_dict[dudvdw_name][:,:])
        output.append(toout)
        del temp
        gc.collect()
        print(toout)
    return output

In [ ]:
Uvar = output_varexplained('W','dw',pca_dict,flatvar_dict,dudvdw_dict,np.linspace(0,34,18))#np.linspace(0,26,14)

0.0
0.039352287
0.1025091
0.14543414
0.17766559
0.21379414
0.25180483
0.2863264
0.30951533
0.3385208
0.37161696


In [ ]:
read_and_proc.save_to_pickle('./W_var_new',Uvar)

#### Functions

In [3]:
#Function####################################################
import json,pickle
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values

def standardize(t,formula='I'):
    if formula=='I':
        return (t-np.min(np.asarray(t)))/(np.max(np.asarray(t))-np.min(np.asarray(t)))
    elif formula=='II':
        return (t-np.mean(np.asarray(t)))/(np.std(np.asarray(t)))
    
def output_PCAtimeseries(PCAdict=None,varname=None,flatvar=None,no_comp=None,standard=True,standardtype='II'):
    if varname is None:
        varname=['dthdt','U','V','W','QV']
    timeseries_out = {}
    for index,var in enumerate(varname):
        if standard is True:
            timeseries_out[var] = standardize(PCAdict[var].transform(flatvar[var])[:,0:no_comp[index]],standardtype)
        else:
            timeseries_out[var] = PCAdict[var].transform(flatvar[var])[:,0:no_comp[index]]
    return timeseries_out

def output_PCAtimeseries_decompose(PCAdict=None,varname=None,flatvar=None,no_comp=None,standard=True,standardtype='II',\
                                   varname_total=['dtheta'],flatvar_store=None):
    """
    Heat decomposition
    """
    if varname is None:
        varname=['dthdt','U','V','W','QV']
    timeseries_out = {}
    for index,var in enumerate(varname):
        if standard is True:
            if var==varname_total[0]:
                storevar = PCAdict[var].transform(flatvar_store[0])[:,0:no_comp[index]]
                mean_store,std_store = np.mean(np.asarray(storevar)),np.std(np.asarray(storevar))
                timeseries_out[var] = PCAdict[var].transform(flatvar[index])[:,0:no_comp[index]]/std_store
                timeseries_mean = mean_store/std_store
            else:
                timeseries_out[var] = standardize(PCAdict[var].transform(flatvar[index])[:,0:no_comp[index]],standardtype)
    return timeseries_out,timeseries_mean

def produce_input(PCAtimeseries=None):
    dtuvw = np.concatenate([y for y in [PCAtimeseries['U'],PCAtimeseries['V'],PCAtimeseries['W'],PCAtimeseries['dthdt']]],axis=1)
    uv = np.concatenate([y for y in [PCAtimeseries['U'],PCAtimeseries['V']]],axis=1)
    dtuv = np.concatenate([y for y in [PCAtimeseries['U'],PCAtimeseries['V'],PCAtimeseries['dthdt']]],axis=1)
    uvw = np.concatenate([y for y in [PCAtimeseries['U'],PCAtimeseries['V'],PCAtimeseries['W']]],axis=1)
    uvwqv = np.concatenate([y for y in [PCAtimeseries['U'],PCAtimeseries['V'],PCAtimeseries['W'],PCAtimeseries['QV']]],axis=1)
    uvqv = np.concatenate([y for y in [PCAtimeseries['U'],PCAtimeseries['V'],PCAtimeseries['QV']]],axis=1)
    dtuvwqv = np.concatenate([y for y in [PCAtimeseries['U'],PCAtimeseries['V'],PCAtimeseries['W'],PCAtimeseries['dthdt'],PCAtimeseries['QV']]],axis=1)
    output_dict={'uv':uv,'uvw':uvw,'uvqv':uvqv,'uvwqv':uvwqv,'dtuvw':dtuvw,'dtuvwqv':dtuvwqv,'dtuv':dtuv}
    return output_dict

In [4]:
def produce_output_LT(PCAtimeseries=None,expname=['ctrl','ncrf36','ncrf60','lwcrf'],orig=True,leadtime=None,nocomp=None):
    def output_timediff(LT=None,inputdict=None,settings=None,exp=None):
        if exp=='ctrl':
            a = prepare_inputdataset2([forward_diff(inputdict['u'],60*60,0,LT),forward_diff(inputdict['v'],60*60,0,LT),forward_diff(inputdict['w'],60*60,0,LT)],np.zeros((forward_diff(inputdict['u'],60*60,0,LT).shape[0],nocomp[1]+nocomp[2]+nocomp[3])),ctrl_t2)
            azero = np.zeros((LT,nocomp[1]+nocomp[2]+nocomp[3]))
            return np.concatenate((a,azero),axis=0)
        elif (exp=='ncrf36') or (exp=='lwcrf'):
            a = prepare_inputdataset2([forward_diff(inputdict['u'][36-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['v'][36-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['w'][36-ctrl_t2:],60*60,0,LT)],
                                        np.zeros((forward_diff(inputdict['u'][36-ctrl_t2:],60*60,0,LT).shape[0],nocomp[1]+nocomp[2]+nocomp[3])),ctrl_t2)
            azero = np.zeros((LT,nocomp[1]+nocomp[2]+nocomp[3]))
            return np.concatenate((a,azero),axis=0)
        elif exp=='ncrf60':
            a = prepare_inputdataset2([forward_diff(inputdict['u'][60-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['v'][60-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['w'][60-ctrl_t2:],60*60,0,LT)],
                                        np.zeros((forward_diff(inputdict['u'][60-ctrl_t2:],60*60,0,LT).shape[0],nocomp[1]+nocomp[2]+nocomp[3])),ctrl_t2)
            azero = np.zeros((LT,nocomp[1]+nocomp[2]+nocomp[3]))
            return np.concatenate((a,azero),axis=0)

    if orig is True:
        result = []
        for index,expNAME in enumerate(expname):
            result.append(output_timediff(LT=leadtime,inputdict=PCAtimeseries[index],settings=None,exp=expNAME))
        del index,expNAME
        result_con = np.concatenate((result),axis=0)
        return result_con

In [ ]:
def prepare_inputdataset(inputTS=None,flat_out=np.zeros((83,12+3+3+9)),lefttimelim=None):
    def flattenlist(inlist=None):
        tempvarlist = []
        for item in inlist:
            tempvarlist.append(item)
        return read_and_proc.flatten(tempvarlist)
    input_dataset36 = []
    for timeseries in inputTS:#[pca_timeseries_36,pcaur_timeseries_36,pcavr_timeseries_36,pcaw_timeseries_36]:
        input_dataset36.append(timeseries)
    for i in (range(inputTS[0][:,0].shape[0])):
        tempinlist = [obj[i,:] for obj in input_dataset36]
        flat_out[i,:] = flattenlist(tempinlist)
    del timeseries,i
    return flat_out

In [ ]:
def prepare_inputdataset(inputTS=None,flat_out=np.zeros((83,12+3+3+9)),lefttimelim=None):
    def flattenlist(inlist=None):
        tempvarlist = []
        for item in inlist:
            tempvarlist.append(item)
        return read_and_proc.flatten(tempvarlist)
    input_dataset36 = []
    for timeseries in inputTS:#[pca_timeseries_36,pcaur_timeseries_36,pcavr_timeseries_36,pcaw_timeseries_36]:
        input_dataset36.append(timeseries)
    for i in (range(inputTS[0][:,0].shape[0])):
        tempinlist = [obj[i,:] for obj in input_dataset36]
        flat_out[i,:] = flattenlist(tempinlist)
    del timeseries,i
    return flat_out

def produce_testdataset(TYPE='orig',LT=1,expTYPE='R21'):
    if expTYPE=='R21':
        if TYPE=='TCr':
            output1_TCr = np.concatenate((prepare_inputdataset([forward_diff(pcaur_timeseries,60*60,0,LT),forward_diff(pcavr_timeseries,60*60,0,LT),forward_diff(pcaw_timeseries,60*60,0,LT)],np.zeros((96,4+3+9)),ctrl_t2),
                                          prepare_inputdataset([forward_diff(pcaur_timeseries_36[36-ctrl_t2-5:],60*60,0,LT),forward_diff(pcavr_timeseries_36[36-ctrl_t2-5:],60*60,0,LT),forward_diff(pcaw_timeseries_36[36-ctrl_t2:],60*60,0,LT)],np.zeros((83,4+3+9)),ctrl_t2),
                                          prepare_inputdataset([forward_diff(pcaur_timeseries_60[60-ctrl_t2-5:],60*60,0,LT),forward_diff(pcavr_timeseries_60[60-ctrl_t2-5:],60*60,0,LT),forward_diff(pcaw_timeseries_60[60-ctrl_t2:],60*60,0,LT)],np.zeros((59,4+3+9)),ctrl_t2),
                                          prepare_inputdataset([forward_diff(pcaur_timeseries_lwcrf[36-ctrl_t2-5:],60*60,0,LT),forward_diff(pcavr_timeseries_lwcrf[36-ctrl_t2-5:],60*60,0,LT),forward_diff(pcaw_timeseries_lwcrf[36-ctrl_t2:],60*60,0,LT)],np.zeros((83,4+3+9)),ctrl_t2)),axis=0)
            return output1_TCr
        elif TYPE=='orig':
            output1_orig = np.concatenate((prepare_inputdataset([forward_diff(pcau_timeseries,60*60,0,LT),forward_diff(pcav_timeseries,60*60,0,LT),forward_diff(pcaw_timeseries,60*60,0,LT)],np.zeros((96,3+3+9)),ctrl_t2),
                                           prepare_inputdataset([forward_diff(pcau_timeseries_36[36-ctrl_t2:],60*60,0,LT),forward_diff(pcav_timeseries_36[36-28:],60*60,0,LT),forward_diff(pcaw_timeseries_36[36-ctrl_t2:],60*60,0,LT)],np.zeros((83,3+3+9)),ctrl_t2),
                                           prepare_inputdataset([forward_diff(pcau_timeseries_60[60-ctrl_t2:],60*60,0,LT),forward_diff(pcav_timeseries_60[60-28:],60*60,0,LT),forward_diff(pcaw_timeseries_60[60-ctrl_t2:],60*60,0,LT)],np.zeros((59,3+3+9)),ctrl_t2),
                                           prepare_inputdataset([forward_diff(pcau_timeseries_lwcrf[36-ctrl_t2:],60*60,0,LT),forward_diff(pcav_timeseries_lwcrf[36-28:],60*60,0,LT),forward_diff(pcaw_timeseries_lwcrf[36-ctrl_t2:],60*60,0,LT)],np.zeros((83,3+3+9)),ctrl_t2)),axis=0)
            return output1_orig

In [ ]:
def output_PCAtimeseries_decompose(PCAdict=None,varname=None,flatvar=None,no_comp=None,standard=True,standardtype='II',\
                                   varname_total=['dtheta'],flatvar_store=None):
    """
    Heat decomposition
    """
    if varname is None:
        varname=['dtheta','u','v','w','qv']
    timeseries_out = {}
    for index,var in enumerate(varname):
        if standard is True:
            if var==varname_total[0]:
                storevar = PCA_dict[var].transform(flatvar_store[0])[:,0:no_comp[index]]
                mean_store,std_store = np.mean(np.asarray(storevar)),np.std(np.asarray(storevar))
                timeseries_out[var] = PCA_dict[var].transform(flatvar[index])[:,0:no_comp[index]]/std_store
                timeseries_mean = mean_store/std_store
            else:
                timeseries_out[var] = standardize(PCA_dict[var].transform(flatvar[index])[:,0:no_comp[index]],standardtype)
    return timeseries_out,timeseries_mean

def produce_input(PCAtimeseries=None,expname='ncrf36',orig=True,leftstart=None,no_comp=None):
    if no_comp is None:
        no_comp=[12,3,3,9,5]
    if expname=='ctrl':
        if orig is True:
            totltime=96
            dtuvw = prepare_inputdataset([PCAtimeseries['u'],PCAtimeseries['v'],PCAtimeseries['w'],PCAtimeseries['dtheta']],
                                         np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3]+no_comp[0])),leftstart)
            uv = prepare_inputdataset([PCAtimeseries['u'],PCAtimeseries['v']],np.zeros((totltime,no_comp[1]+no_comp[2])),leftstart)
            dtuv = prepare_inputdataset([PCAtimeseries['u'],PCAtimeseries['v'],PCAtimeseries['dtheta']],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[0])),leftstart)
            uvw = prepare_inputdataset([PCAtimeseries['u'],PCAtimeseries['v'],PCAtimeseries['w']],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3])),leftstart)
            uvwqv = prepare_inputdataset([PCAtimeseries['u'],PCAtimeseries['v'],PCAtimeseries['w'],PCAtimeseries['qv']],
                                     np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3]+no_comp[4])),leftstart)
            uvqv = prepare_inputdataset([PCAtimeseries['u'],PCAtimeseries['v'],PCAtimeseries['qv']],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[4])),leftstart)
            dtuvwqv = prepare_inputdataset([PCAtimeseries['u'],PCAtimeseries['v'],PCAtimeseries['w'],PCAtimeseries['dtheta'],PCAtimeseries['qv']],
                                           np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3]+no_comp[0]+no_comp[4])),leftstart)
            output_dict={'uv':uv,'uvw':uvw,'uvqv':uvqv,'uvwqv':uvwqv,'dtuvw':dtuvw,'dtuvwqv':dtuvwqv,'dtuv':dtuv}
            return output_dict
    else:
        if expname=='ncrf36':
            rsindx,totltime = 36,96-(36-23)
        elif expname=='ncrf60':
            rsindx,totltime = 60,96-(60-23)
        
        if orig is True:
            dtuvw = prepare_inputdataset([PCAtimeseries['u'][rsindx-leftstart:],PCAtimeseries['v'][rsindx-leftstart:],
                                          PCAtimeseries['w'][rsindx-leftstart:],PCAtimeseries['dtheta'][rsindx-leftstart:]],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3]+no_comp[0])),leftstart)
            uv = prepare_inputdataset([PCAtimeseries['u'][rsindx-leftstart:],PCAtimeseries['v'][rsindx-leftstart:]],np.zeros((totltime,no_comp[1]+no_comp[2])),leftstart)
            uvw = prepare_inputdataset([PCAtimeseries['u'][rsindx-leftstart:],PCAtimeseries['v'][rsindx-leftstart:],PCAtimeseries['w'][rsindx-leftstart:]],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3])),leftstart)
            dtuv = prepare_inputdataset([PCAtimeseries['u'][rsindx-leftstart:],PCAtimeseries['v'][rsindx-leftstart:],PCAtimeseries['dtheta'][rsindx-leftstart:]],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[0])),leftstart)
            uvwqv = prepare_inputdataset([PCAtimeseries['u'][rsindx-leftstart:],PCAtimeseries['v'][rsindx-leftstart:],
                                          PCAtimeseries['w'][rsindx-leftstart:],PCAtimeseries['qv'][rsindx-leftstart:]],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3]+no_comp[4])),leftstart)
            uvqv = prepare_inputdataset([PCAtimeseries['u'][rsindx-leftstart:],PCAtimeseries['v'][rsindx-leftstart:],PCAtimeseries['qv'][rsindx-leftstart:]],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[4])),leftstart)
            dtuvwqv = prepare_inputdataset([PCAtimeseries['u'][rsindx-leftstart:],PCAtimeseries['v'][rsindx-leftstart:],
                                            PCAtimeseries['w'][rsindx-leftstart:],PCAtimeseries['dtheta'][rsindx-leftstart:],
                                            PCAtimeseries['qv'][rsindx-leftstart:]],np.zeros((totltime,no_comp[1]+no_comp[2]+no_comp[3]+no_comp[0]+no_comp[4])),leftstart)
            output_dict={'uv':uv,'uvw':uvw,'uvqv':uvqv,'uvwqv':uvwqv,'dtuvw':dtuvw,'dtuvwqv':dtuvwqv,'dtuv':dtuv}
            return output_dict

def produce_output_LT(PCAtimeseries=None,expname=['ctrl','ncrf36','ncrf60','lwcrf'],orig=True,leadtime=None,nocomp=None):
    def output_timediff(LT=None,inputdict=None,settings=None,exp=None):
        if exp=='ctrl':
            a = prepare_inputdataset2([forward_diff(inputdict['u'],60*60,0,LT),forward_diff(inputdict['v'],60*60,0,LT),forward_diff(inputdict['w'],60*60,0,LT)],np.zeros((forward_diff(inputdict['u'],60*60,0,LT).shape[0],nocomp[1]+nocomp[2]+nocomp[3])),ctrl_t2)
            azero = np.zeros((LT,nocomp[1]+nocomp[2]+nocomp[3]))
            return np.concatenate((a,azero),axis=0)
        elif (exp=='ncrf36') or (exp=='lwcrf'):
            a = prepare_inputdataset2([forward_diff(inputdict['u'][36-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['v'][36-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['w'][36-ctrl_t2:],60*60,0,LT)],
                                        np.zeros((forward_diff(inputdict['u'][36-ctrl_t2:],60*60,0,LT).shape[0],nocomp[1]+nocomp[2]+nocomp[3])),ctrl_t2)
            azero = np.zeros((LT,nocomp[1]+nocomp[2]+nocomp[3]))
            return np.concatenate((a,azero),axis=0)
        elif exp=='ncrf60':
            a = prepare_inputdataset2([forward_diff(inputdict['u'][60-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['v'][60-ctrl_t2:],60*60,0,LT),forward_diff(inputdict['w'][60-ctrl_t2:],60*60,0,LT)],
                                        np.zeros((forward_diff(inputdict['u'][60-ctrl_t2:],60*60,0,LT).shape[0],nocomp[1]+nocomp[2]+nocomp[3])),ctrl_t2)
            azero = np.zeros((LT,nocomp[1]+nocomp[2]+nocomp[3]))
            return np.concatenate((a,azero),axis=0)

    if orig is True:
        result = []
        for index,expNAME in enumerate(expname):
            result.append(output_timediff(LT=leadtime,inputdict=PCAtimeseries[index],settings=None,exp=expNAME))
        del index,expNAME
        result_con = np.concatenate((result),axis=0)
        return result_con
    
def prepare_inputdataset2(inputTS=None,flat_out=np.zeros((83,12+3+3+9)),lefttimelim=None):
    def flattenlist(inlist=None):
        tempvarlist = []
        for item in inlist:
            tempvarlist.append(item)
        return read_and_proc.flatten(tempvarlist)
    input_dataset36 = []
    for timeseries in inputTS:#[pca_timeseries_36,pcaur_timeseries_36,pcavr_timeseries_36,pcaw_timeseries_36]:
        input_dataset36.append(timeseries)
    for i in (range(inputTS[0][:,0].shape[0])):
        tempinlist = [obj[i,:] for obj in input_dataset36]
        flat_out[i,:] = flattenlist(tempinlist)
    del timeseries,i
    return flat_out
        
#    produce_testdataset(TYPE='TCr',LT=1,expTYPE='R21'),produce_testdataset(TYPE='TCr',LT=2,expTYPE='R21'),produce_testdataset(TYPE='TCr',LT=3,expTYPE='R21')

#### Produce inputs

In [5]:
origvar_dict = {}
for inx,obj in tqdm(enumerate(['U'])):
    origvar_dict[obj] = read_and_proc.depickle(path+'origvar/'+str(obj)+'_origvar')

no_comp = [13,14,15,38,4]
PCAtimeseries = output_PCAtimeseries(PCAdict=pca_dict,varname=['dthdt','U','V','W','QV'],flatvar=flatvar_dict,no_comp=no_comp,standard=True,standardtype='II')

1it [00:10, 10.20s/it]


In [6]:
mlr_inputdict = produce_input(PCAtimeseries)

In [7]:
def divide_longseries(timeseries=None,indices=[96,96+84,96+84+60,96+84+60+84],no_comp=[14,15,38],varname='U'):
    if varname=='U':
        output = {'ctl':timeseries[0:indices[0],0:int(no_comp[0])],'ncrf36':timeseries[indices[0]:indices[1],0:int(no_comp[0])],\
                  'ncrf60':timeseries[indices[1]:indices[2],0:int(no_comp[0])],'lwcrf':timeseries[indices[2]:indices[3],0:int(no_comp[0])]}
    elif varname=='V':
        output = {'ctl':timeseries[0:indices[0],0:int(no_comp[1])],'ncrf36':timeseries[indices[0]:indices[1],0:int(no_comp[1])],\
                  'ncrf60':timeseries[indices[1]:indices[2],0:int(no_comp[1])],'lwcrf':timeseries[indices[2]:indices[3],0:int(no_comp[1])]}
    elif varname=='W':
        output = {'ctl':timeseries[0:indices[0],0:int(no_comp[2])],'ncrf36':timeseries[indices[0]:indices[1],0:int(no_comp[2])],\
                  'ncrf60':timeseries[indices[1]:indices[2],0:int(no_comp[2])],'lwcrf':timeseries[indices[2]:indices[3],0:int(no_comp[2])]}
    return output
Udict = divide_longseries(PCAtimeseries['U'],indices=[96,96+84,96+84+60,96+84+60+84],no_comp=[14,15,38],varname='U')
Vdict = divide_longseries(PCAtimeseries['V'],indices=[96,96+84,96+84+60,96+84+60+84],no_comp=[14,15,38],varname='V')
Wdict = divide_longseries(PCAtimeseries['W'],indices=[96,96+84,96+84+60,96+84+60+84],no_comp=[14,15,38],varname='W')

In [39]:
import preproc_noensemble
def produce_tooutput(WINDdict=None,LT=np.linspace(1,36,36)):
    storedict = {}
    for inx,obj in enumerate(['ctl','ncrf36','ncrf60','lwcrf']):
        storedict[obj] = [preproc_noensemble.forward_diff(WINDdict[obj],60*60,0,int(LTi)) for LTi in LT]
    return storedict
Ustoredict = produce_tooutput(Udict,np.linspace(1,36,36))
Vstoredict = produce_tooutput(Vdict,np.linspace(1,36,36))
Wstoredict = produce_tooutput(Wdict,np.linspace(1,36,36))

In [45]:
Ustoredict['ctl'][0].shape,Ustoredict['ncrf36'][0].shape,Ustoredict['ncrf60'][0].shape,Ustoredict['lwcrf'][0].shape

((95, 14), (83, 14), (59, 14), (82, 14))

In [48]:
def produce_tooutput2(storedict=None):
    expname = list(storedict.keys())
    output = []
    for i in tqdm(range(len(storedict[expname[0]]))):
        TEMP = [storedict[exps][i] for exps in expname]
        temp = np.concatenate([y for y in TEMP])
        output.append(temp)
    return output
Upreoutput = produce_tooutput2(Ustoredict)
Vpreoutput = produce_tooutput2(Vstoredict)
Wpreoutput = produce_tooutput2(Wstoredict)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 6843.81it/s]


In [56]:
mlr_output = []
for i in range(len(Upreoutput)):
    temp=[Upreoutput[i],Vpreoutput[i],Wpreoutput[i]]
    mlr_output.append(np.concatenate([y for y in temp],axis=1))

In [57]:
mlr_output[0].shape,mlr_output[10].shape

((319, 67), (279, 67))

In [58]:
read_and_proc.save_to_pickle('/scratch/08350/tg876493/ruppert2020/output/mlr_output',mlr_output) 

In [19]:
indices = read_and_proc.depickle('/scratch/08350/tg876493/ruppert2020/output/index_array')

#### Test

In [ ]:
######################################################################################################################################
# PCA timeseries
#.....................................................................................................................
no_comp = [12,11,11,15,5+1] #80%
# Radiation ...................................................................................................................................................................................................................................
#.....................................................................................................................
ctrl_PCAtimeseriesO,ctrl_mean = output_PCAtimeseries_decompose(PCAdict=PCA_dict,varname=['dtheta','u','v','w','qv'],
                                           flatvar=[NORAD_Afdict['ctrlRESI'],Af_dict['u'],Af_dict['v'],Af_dict['w'],Af_dict['qv']],
                                           no_comp=no_comp,standard=True,standardtype='II',varname_total=['dtheta'],flatvar_store=[HEAT_Afdict['ctrlHEAT']])
ncrf36_PCAtimeseriesO,ncrf36_mean = output_PCAtimeseries_decompose(PCAdict=PCA_dict,varname=['dtheta','u','v','w','qv'],
                                             flatvar=[NORAD_Afdict['ncrf36RESI'],normalUradncrf36f,normalVtanncrf36f,normalWncrf36f,normalWncrf36f],
                                             no_comp=no_comp,standard=True,standardtype='II',varname_total=['dtheta'],flatvar_store=[HEAT_Afdict['ncrf36HEAT']])
ncrf60_PCAtimeseriesO,ncrf60_mean = output_PCAtimeseries_decompose(PCAdict=PCA_dict,varname=['dtheta','u','v','w','qv'],
                                             flatvar=[NORAD_Afdict['ncrf60RESI'],normalUradncrf60f,normalVtanncrf60f,normalWncrf60f,normalWncrf60f],
                                             no_comp=no_comp,standard=True,standardtype='II',varname_total=['dtheta'],flatvar_store=[HEAT_Afdict['ncrf60HEAT']])
lwcrf_PCAtimeseriesO,lwcrf_mean = output_PCAtimeseries_decompose(PCAdict=PCA_dict,varname=['dtheta','u','v','w','qv'],
                                            flatvar=[NORAD_Afdict['lwcrfRESI'],normalUradlwcrff,normalVtanlwcrff,normalWlwcrff,normalWlwcrff],
                                            no_comp=no_comp,standard=True,standardtype='II',varname_total=['dtheta'],flatvar_store=[HEAT_Afdict['lwcrfHEAT']])
######################################################################################################################################
# Produce Input
#.....................................................................................................................
#CTRLinputdataset_dict = produce_input(ctrl_PCAtimeseriesO,expname='ctrl',orig=True,leftstart=None,no_comp=no_comp)
#NCRF36inputdataset_dict = produce_input(ncrf36_PCAtimeseriesO,expname='ncrf36',orig=True,leftstart=ctrl_t2,no_comp=no_comp)
#NCRF60inputdataset_dict = produce_input(ncrf60_PCAtimeseriesO,expname='ncrf60',orig=True,leftstart=ctrl_t2,no_comp=no_comp)
#LWCRFinputdataset_dict = produce_input(lwcrf_PCAtimeseriesO,expname='ncrf36',orig=True,leftstart=ctrl_t2,no_comp=no_comp)

# Produce Fix Input
#.....................................................................................................................
print(ctrl_mean,ncrf36_mean,ncrf60_mean,lwcrf_mean)
######################################################################################################################################

In [ ]:
######################################################################################################################################
#.....................................................................................................................
# Make input digestable with our MLR model
#.....................................................................................................................
mlr_inputtype = ['uv','uvw','uvqv','dtuv','uvwqv','dtuvw','dtuvwqv']
mlr_inputdict = {}
for TYPE in mlr_inputtype:
    result = np.concatenate([exp[TYPE] for exp in [CTRLinputdataset_dict,NCRF36inputdataset_dict,NCRF60inputdataset_dict,LWCRFinputdataset_dict]],axis=0)
    mlr_inputdict[TYPE] = result
#.....................................................................................................................
# MLR model output for training
#.....................................................................................................................
LDTME = np.linspace(0,44,45)+1
mlr_output = []
for index,TIMEstamp in enumerate(LDTME):
    mlr_output.append(produce_output_LT([ctrl_PCAtimeseriesO,ncrf36_PCAtimeseriesO,ncrf60_PCAtimeseriesO,lwcrf_PCAtimeseriesO],['ctrl','ncrf36','ncrf60','lwcrf'],True,leadtime=int(TIMEstamp),nocomp=no_comp))
del CTRLinputdataset_dict,NCRF36inputdataset_dict,NCRF60inputdataset_dict,LWCRFinputdataset_dict

### Multilinear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import MultiTaskLassoCV
from celer import MultiTaskLassoCV
from celer import Lasso
from sklearn.linear_model import ElasticNet
def produce_regscore(inPUT=None,outPUT=None,aux_inPUT=None,outtype='score',do_aux=False,
                     algorithm='linear',lassoparam=dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),simplelassoalpha=0.001):
    if algorithm=='linear':
        reg = LinearRegression().fit(inPUT, outPUT)
    elif algorithm=='lasso':
        #reg = MultiTaskLassoCV(**lassoparam).fit(inPUT,outPUT)
        reg = Lasso(simplelassoalpha).fit(inPUT,outPUT)
    elif algorithm=='multilasso':
        reg = MultiTaskLassoCV(**lassoparam).fit(inPUT,outPUT)
    if outtype=='score':
        if do_aux is False:
            return reg.score(inPUT, outPUT)
        elif do_aux is True:
            return reg.score(aux_inPUT,outPUT)
    elif outtype=='predict':
        return reg.predict(inPUT)
    elif outtype=='coeff':
        return reg.coef_

def delete_padding(inTS=None,outTS=None):
    output_nozero,input_nozero = [],[]
    for i in range(len(outTS[:,0])):
        temp = outTS[i,:]
        tempin = inTS[i,:]
        if temp.all()==0:
            continue
        else:
            output_nozero.append(temp)
            input_nozero.append(tempin)
    return input_nozero,output_nozero

def output_regscore(inTS=None,outTS=None,LTlist=None,algorithm='linear',aux_inTS=None,do_aux=False,simplelassoalpha=0.001):
    if do_aux is False:
        return [produce_regscore(delete_padding(inTS,outTS[i])[0],delete_padding(inTS,outTS[i])[1],None,'score',False,algorithm,\
                                 dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),simplelassoalpha) for i in range(len(outTS))]
    elif do_aux is True:
        return [produce_regscore(delete_padding(inTS,outTS[i])[0],delete_padding(inTS,outTS[i])[1],aux_inTS[:-int(LTlist[i])],'score',True,algorithm,\
                                 dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),simplelassoalpha) for i in range(len(outTS))]
    
def output_regscore_withmemory(inTS=None,outTS=None,LTlist=None,algorithm='linear',aux_inTS=None,do_aux=False,simplelassoalpha=0.001):
    if do_aux is False:
        output = []
        for i in (range(len(outTS))):
            inTSn,outTSn = delete_padding(inTS,outTS[i])[0],delete_padding(inTS,outTS[i])[1]
            inTRAIN = np.concatenate(((inTSn[int(LTlist[i]):]),(inTSn[0:-int(LTlist[i])])),axis=1)
            outTRAIN = outTSn[int(LTlist[i]):]
            output.append(produce_regscore(inTRAIN,outTRAIN,None,'score',False,algorithm,dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),simplelassoalpha))
        return output
    elif do_aux is True:
        output = []
        for i in (range(len(outTS))):
            inTSn,outTSn = delete_padding(inTS,outTS[i])[0],delete_padding(inTS,outTS[i])[1]
            inTRAIN = np.concatenate(((inTSn[int(LTlist[i]):]),(inTSn[0:-int(LTlist[i])])),axis=1)
            outTRAIN = outTSn[int(LTlist[i])]
            auxin = np.concatenate(((aux_inTS[:-int(LTlist[i])][int(LTlist[i]):]),(aux_inTS[:-int(LTlist[i])][0:-int(LTlist[i])])),axis=1)
            output.append(produce_regscore(inTRAIN,outTRAIN,auxin,'score',True,algorithm,dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),simplelassoalpha))
        return output        
    
def output_regcoeff(inTS=None,outTS=None,LTlist=None,algorithm='linear',memory='with',simplelassoalpha=0.001):
    if memory=='with':
        output = []
        for i in range(len(outTS)):
            inarray = np.concatenate(((inTS[:-int(LTlist[i])][int(LTlist[i]):]),(inTS[:-int(LTlist[i])][0:-int(LTlist[i])])),axis=1)
            outarray = outTS[i][int(LTlist[i]):-int(LTlist[i])]
            output.append(produce_regscore(inarray,outarray,'coeff',algorithm,dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),0.01))
        return output
    elif memory=='without':
        return [produce_regscore(inTS[:-int(LTlist[i])],outTS[i][:-int(LTlist[i])],'coeff',algorithm,dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),simplelassoalpha) for i in range(len(outTS))]
    
def output_regpredict(inTS=None,outTS=None,LTlist=None,algorithm='linear',memory='without',simplelassoalpha=0.001):
    if memory=='with':
        output = []
        for i in range(len(outTS)):
            inarray = np.concatenate(((inTS[:-int(LTlist[i])][int(LTlist[i]):]),(inTS[:-int(LTlist[i])][0:-int(LTlist[i])])),axis=1)
            outarray = outTS[i][int(LTlist[i]):-int(LTlist[i])]
            output.append(produce_regscore(inarray,outarray,None,'predict',False,algorithm,dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),0.01))
        return output
    elif memory=='without':
        return [produce_regscore(delete_padding(inTS,outTS[i])[0],delete_padding(inTS,outTS[i])[1],None,'predict',False,algorithm,
                                 dict(tol=1e-6,cv=4,n_jobs=1,n_alphas=20),simplelassoalpha) for i in range(len(outTS))]

In [ ]:
r2rad_timeseries = []
with open('./output/1010_ns/sensitivity/r2rad_timeseries','rb') as f:
    r2rad_timeseries.append(pickle.load(f))
rad_input = []
with open('./output/1010_ns/sensitivity/r2rad_input','rb') as f:
    rad_input.append(pickle.load(f))
rad_output = []
with open('./output/1010_ns/sensitivity/r2rad_output','rb') as f:
    rad_output.append(pickle.load(f))
    
r2resi_timeseries = []
with open('./output/1010_ns/sensitivity/r2resi_timeseries','rb') as f:
    r2resi_timeseries.append(pickle.load(f))
resi_input = []
with open('./output/1010_ns/sensitivity/r2resi_input','rb') as f:
    resi_input.append(pickle.load(f))
resi_output = []
with open('./output/1010_ns/sensitivity/r2resi_output','rb') as f:
    resi_output.append(pickle.load(f))

In [ ]:
fig,axs = plot.subplots(ncols=2,nrows=1,axwidth=3.55,refaspect=2.051)
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                  simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v',c='k')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvw'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                  simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-w')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                  simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-dtheta')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvqv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                  simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-qv')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuvw'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                  simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='dtheta-u-v-w')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                  simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='dtheta-u-v-w-qv')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                  simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-w-qv')
axs[0].legend(loc='b')
axs[0].format(ylabel='R$^2$',xlabel='Lead Time [Hr]',title='orig-orig')

axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                             simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v',c='k')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvw'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                             simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-w')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                             simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-dtheta')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvqv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                             simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-qv')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuvw'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                             simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='dtheta-u-v-w')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                             simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='dtheta-u-v-w-qv')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='lasso',
                                             simplelassoalpha=0.001,aux_inTS=None,do_aux=False),label='u-v-w-qv')
axs[1].legend(loc='b')
axs[1].format(ylabel='R$^2$',xlabel='Lead Time [Hr]',title='orig-orig [Memory]')
#plt.savefig('./proc_fig/1129/r2long_orig_moreterms.png',dpi=400)
plt.show()

In [ ]:
fig,axs = plot.subplots(ncols=2,nrows=1,axwidth=3.55,refaspect=2.051)
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v',c='k')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-dtheta')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-qv')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w-qv')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w-qv')
axs[0].legend(loc='b')
axs[0].format(ylabel='R$^2$',xlabel='Lead Time [Hr]',title='orig-orig')

axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v',c='k')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-dtheta')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-qv')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w-qv')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w-qv')
axs[1].legend(loc='b')
axs[1].format(ylabel='R$^2$',xlabel='Lead Time [Hr]',title='orig-orig [Memory]')
plt.savefig('./proc_fig/1129/r2long_orig_moreterms_T6.png',dpi=400)
plt.show()

In [ ]:
fig,axs = plot.subplots(ncols=2,nrows=1,axwidth=3.55,refaspect=2.051)
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v',c='k')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-dtheta')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-qv')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['dtuvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w-qv')
axs[0].plot(LDTME,output_regscore(inTS=mlr_inputdict['uvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w-qv')
axs[0].legend(loc='b')
axs[0].format(ylabel='R$^2$',xlabel='Lead Time [Hr]',title='orig-orig')

axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v',c='k')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-dtheta')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-qv')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuvw'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['dtuvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='dtheta-u-v-w-qv')
axs[1].plot(LDTME,output_regscore_withmemory(inTS=mlr_inputdict['uvwqv'],outTS=mlr_output,LTlist=LDTME,algorithm='linear',aux_inTS=None,do_aux=False),label='u-v-w-qv')
axs[1].legend(loc='b')
axs[1].format(ylabel='R$^2$',xlabel='Lead Time [Hr]',title='orig-orig [Memory]')
plt.savefig('./proc_fig/1129/r2long_orig_moreterms_T1_norm.png',dpi=400)
plt.show()

In [ ]:
fig,axs = plot.subplots(ncols=1,nrows=1,axwidth=3.55,refaspect=1.751,share=False)
leadtimec = 1
cout=axs[0].pcolormesh(LinearRegression().fit(mlr_inputdict['dtuvw'][:-int(leadtimec)], 
                                              mlr_output[int(leadtimec)-1][:-int(leadtimec)]).coef_/1e-5,cmap='Div',cmap_kw={'cut': -0.01},
                       vmin=-4,vmax=4,edgecolors='k', linewidths=0.25,alpha=0.99, shading='auto')
axs[0].colorbar(cout,loc='r')
axs[0].format(xticks=[1.15,3.85,9.75,20.5],xticklabels=['u$_{rad,r}$','v$_{tan,r}$','w',r'd$\theta$/dt'],yticks=[1.5,4.5,9.5],
              yticklabels=['du$_{rad}$/dt','dv$_{tan}$/dt','dw/dt'],title='Lead Time: 1hr; orig; No Memory [Residual]')
axs[0].tick_params(axis='y',which='both',left=False)
axs[0].tick_params(axis='x',which='both',bottom=False)
for i in [3-0.5,6-0.5,16-0.5,16-0.5+12]:
    axs[0].axvline(i,lw=3,color='k')
for i in [3-0.5,6-0.5]:
    axs[0].axhline(i,lw=3,color='k')
#plt.savefig('./proc_fig/1122/uvwt_lt6_nm_coeff_resiheat.png',dpi=400)
plt.show()

In [ ]:
import matplotlib.ticker as ticker
def fmt(x, pos):
    a, b = '{:.2e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

In [ ]:
fig,axs = plot.subplots(ncols=2,nrows=1,axwidth=3.55,refaspect=1.751,share=False)
leadtimec = 6
cout=axs[0].pcolormesh(LinearRegression().fit(mlr_inputdict['dtuvw'][:-int(leadtimec)], 
                                              mlr_output[int(leadtimec)-1][:-int(leadtimec)]).coef_,cmap='Div',cmap_kw={'cut': -0.01},
                       vmin=-4e-5,vmax=4e-5,edgecolors='k', linewidths=0.25,alpha=0.99, shading='auto')
cbar = axs[0].colorbar(cout,loc='b',format=ticker.FuncFormatter(fmt))
#cbar.formatter.set_powerlimits((0, 0))
axs[0].format(xticks=[1.15,3.85,9.75,20.5],xticklabels=['u$_{rad,r}$','v$_{tan,r}$','w',r'd$\theta$/dt'],yticks=[1.5,4.5,9.5],
              yticklabels=['du$_{rad}$/dt','dv$_{tan}$/dt','dw/dt'],title='Lead Time: 6hr; orig; No Memory [Full Heating]')
axs[0].tick_params(axis='y',which='both',left=False)
axs[0].tick_params(axis='x',which='both',bottom=False)
for i in [3-0.5,6-0.5,15-0.5,15-0.5+12]:
    axs[0].axvline(i,lw=3,color='k')
for i in [3-0.5,6-0.5]:
    axs[0].axhline(i,lw=3,color='k')

cout=axs[1].pcolormesh(LinearRegression().fit(rad_input[0]['dtuvw'][:-int(leadtimec)], 
                                              rad_output[0][int(leadtimec)-1][:-int(leadtimec)]).coef_,cmap='Div',cmap_kw={'cut': -0.01},
                       vmin=-10e-5,vmax=10e-5,edgecolors='k', linewidths=0.25,alpha=0.99, shading='auto')
cbar2 = axs[1].colorbar(cout,loc='b',ticks=[-8e-5,-4e-5,0,4e-5,8e-5],format=ticker.FuncFormatter(fmt))
#cbar2.formatter.set_powerlimits((0, 0))
axs[1].format(xticks=[1.15,3.85,9.75,20.5],xticklabels=['u$_{rad,r}$','v$_{tan,r}$','w',r'd$\theta$/dt'],yticks=[1.5,4.5,9.5],
              yticklabels=['du$_{rad}$/dt','dv$_{tan}$/dt','dw/dt'],title='Lead Time: 6hr; orig; No Memory [Radiative]')
axs[1].tick_params(axis='y',which='both',left=False)
axs[1].tick_params(axis='x',which='both',bottom=False)
for i in [3-0.5,6-0.5,15-0.5,15-0.5+12]:
    axs[1].axvline(i,lw=3,color='k')
for i in [3-0.5,6-0.5]:
    axs[1].axhline(i,lw=3,color='k')
plt.savefig('./proc_fig/1122/uvwt_lt6_nm_coeff_fullheatradheat.png',dpi=400)
plt.show()

### Reconstruction

In [ ]:
import json,pickle
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

save_to_pickle(loc='./output/1010_nc/MLR_train/radiation/simple/mlr_inputdictRESIn_T6',var=mlr_inputdict)
save_to_pickle(loc='./output/1010_nc/MLR_train/radiation/simple/mlr_outputRESIn_T6',var=mlr_output)

In [ ]:
fileA = [path+'normalT_ctrl_nc']
outputA = []
for i in range(len(fileA)):
    with open(fileA[i],'rb') as f:
        outputA.append(pickle.load(f))
A_dict = {'dtheta':outputA[0]}

In [ ]:
forecast_eiginput = output_regpredict(mlr_inputdict['dtuvw'],mlr_output,LDTME,'linear','without')

In [ ]:
def windrates_real(u=[Af_dict['u'],normalUradncrf36f,normalUradncrf60f,normalUradlwcrff],
                   v=[Af_dict['v'],normalVtanncrf36f,normalVtanncrf60f,normalVtanlwcrff],w=[Af_dict['w'],normalWncrf36f,normalWncrf60f,normalWlwcrff],LT=None):
    # dudt
    dudtT = [forward_diff(uobj,60*60,0,LT) for uobj in u]
    dudt = np.concatenate((dudtT[0],dudtT[1][96-(36-23):],dudtT[2][96-(60-23):],dudtT[3][96-(36-23):]),axis=0)
    del dudtT
    # dvdt
    dvdtT = [forward_diff(vobj,60*60,0,LT) for vobj in v]
    dvdt = np.concatenate((dvdtT[0],dvdtT[1][96-(36-23):],dvdtT[2][96-(60-23):],dvdtT[3][96-(36-23):]),axis=0)
    del dvdtT 
    # dwdt
    dwdtT = [forward_diff(wobj,60*60,0,LT) for wobj in w]
    dwdt = np.concatenate((dwdtT[0],dwdtT[1][96-(36-23):],dwdtT[2][96-(60-23):],dwdtT[3][96-(36-23):]),axis=0)
    del dwdtT 
    return dudt,dvdt,dwdt

def recon_from_linear(forecast_eiginput=None,PCA_dict=None):
    def output_reshapeRECON(forecast_eig=None,PCAdict=None):
        testrec_dudt = np.dot(forecast_eig[:,0:3],(PCA_dict['u'].components_[0:3]))#.reshape((91,39,360,167))
        testrec_dvdt = np.dot(forecast_eig[:,3:6],(PCA_dict['v'].components_[0:3]))#.reshape((91,39,360,167))
        testrec_dwdt = np.dot(forecast_eig[:,6:],(PCA_dict['w'].components_[0:10]))#.reshape((39,360,167))
        return testrec_dudt,testrec_dvdt,testrec_dwdt
    ######################################################################################################################################################
    name = ['dudt','dvdt','dwdt']
    temp1,temp2,temp3 = [],[],[]
    temp1b,temp2b,temp3b = [],[],[]
    for i in tqdm(range(45)):
        teMP1,teMP2,teMP3 = output_reshapeRECON(forecast_eiginput[i],PCA_dict)
        reteMP1,reteMP2,reteMP3 = windrates_real(LT=i)
        # Square error
        temp1.append((teMP1-reteMP1)**2)
        temp2.append((teMP2-reteMP2)**2)
        temp3.append((teMP3-reteMP3)**2)
        # Variance
        temp1b.append((teMP1-np.nanmean(teMP1))**2)
        temp2b.append((teMP2-np.nanmean(teMP2))**2)
        temp3b.append((teMP3-np.nanmean(teMP3))**2)
        del teMP1,teMP2,teMP3,rateMP1,reteMP2,reteMP3
    del i
    se_store = {name[0]:temp1,name[1]:temp2,name[2]:temp3}
    va_store = {name[0]:temp1b,name[1]:temp2b,name[2]:temp3b}
    ######################################################################################################################################################
    return recon_store,va_store

In [ ]:
import scipy
def azimuth2angle(azimuth=None):
    """
    https://math.stackexchange.com/questions/926226/conversion-from-azimuth-to-counterclockwise-angle
    """
    angletest = 450-azimuth
    for index,item in enumerate(angletest):
        if item>360:
            angletest[index] = item-360
        else:
            continue
    return angletest

def closest_index(array=None,target=None):
    return np.abs(array-target).argmin()

def polar2cartesian(outcoords, inputshape, origin):
    """Coordinate transform for converting a polar array to Cartesian coordinates. 
    inputshape is a tuple containing the shape of the polar array. origin is a
    tuple containing the x and y indices of where the origin should be in the
    output array."""

    xindex, yindex = outcoords
    x0, y0 = origin
    x = xindex - x0
    y = yindex - y0

    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    theta_index = np.round((theta + np.pi) * inputshape[1] / (2 * np.pi))

    return (r,theta_index)

def proc_tocart(polarfield=None,angle=None,twoD=True,twoDlv=None,standard=False):
    if twoD==False:
        PWnew = [np.asarray(polarfield)[int(twoDlv),int(np.abs(ANGLE-360).argmin()),:]]
        for i in np.linspace(0,358,359):
            PWnew.append(np.asarray(polarfield)[int(twoDlv),int(np.abs(ANGLE-i).argmin()),:])
        PWnew = np.swapaxes(np.asarray(PWnew),0,1)
        del i
        
        if standard==True:
            PWnew = (PWnew-np.nanmean(PWnew))/np.nanstd(PWnew)
        else:
            PWnew=PWnew

        test_2cartesian = scipy.ndimage.geometric_transform(PWnew, 
                                                            polar2cartesian,order=0,mode='constant',
                                                            output_shape = \
                                                            (PWnew.shape[0] * 2,PWnew.shape[0] * 2),
                                                            extra_keywords = \
                                                            {'inputshape':PWnew.shape,
                                                             'origin':(PWnew.shape[0],PWnew.shape[0])})
        #print('Finish processing')
        return ((test_2cartesian))
    elif twoD==True:
        PWnew = [np.asarray(polarfield)[int(np.abs(ANGLE-360).argmin()),:]]
        for i in np.linspace(0,358,359):
            PWnew.append(np.asarray(polarfield)[int(np.abs(ANGLE-i).argmin()),:])
        PWnew = np.swapaxes(np.asarray(PWnew),0,1)
        del i
        
        if standard==True:
            PWnew = (PWnew-np.nanmean(PWnew))/np.nanstd(PWnew)
        else:
            PWnew=PWnew

        test_2cartesian = scipy.ndimage.geometric_transform(PWnew, 
                                                            polar2cartesian,order=0,mode='constant',
                                                            output_shape = \
                                                            (PWnew.shape[0] * 2,PWnew.shape[0] * 2),
                                                            extra_keywords = \
                                                            {'inputshape':PWnew.shape,
                                                             'origin':(PWnew.shape[0],PWnew.shape[0])})
        #print('Finish processing')
        return ((test_2cartesian))

In [ ]:
ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_W_0-144hr.nc'],fieldname=['W'])
############################################################################################################################################
ANGLE = azimuth2angle(np.linspace(0,359,360))
NEWCART_axis = np.concatenate((np.flipud(-ctrlvar_dict['W'].radius[:r500]),ctrlvar_dict['W'].radius[:r500][:]))
del ctrlvar_dict